In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from functools import partial

import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch import nn

import emmental
from emmental import Meta
from emmental.learner import EmmentalLearner
from emmental.model import EmmentalModel
from emmental.scorer import Scorer
from emmental.task import EmmentalTask
from modules.bert_module import BertModule
from parse_CB_slice import get_CB_dataloaders
from task_config import SuperGLUE_LABEL_MAPPING, SuperGLUE_TASK_METRIC_MAPPING

In [3]:
logger = logging.getLogger(__name__)

# Initalize Emmental

In [4]:
emmental.init(
    "logs",
    config={
        "meta_config": {"seed": 111},
        "model_config": {"device": 1, "dataparallel": False},
        "learner_config": {
            "n_epochs": 20,
            "valid_split": "val",
            "optimizer_config": {"optimizer": "adam", "lr": 0.00001},
            "min_lr": 0,
            "lr_scheduler_config": {"warmup_percentage": 0.1, "lr_scheduler": "linear"},
        },
        "logging_config": {
            "counter_unit": "epoch",
            "evaluation_freq": 1,
            "checkpointing": True,
            "checkpointer_config": {
                "checkpoint_metric": {"CB/SuperGLUE/val/accuacy_macro_f1": "max"}
            },
        },
    },
)

[2019-05-22 18:34:00,942][INFO] emmental.meta:95 - Setting logging directory to: logs/2019_05_22/18_34_00
[2019-05-22 18:34:00,960][INFO] emmental.meta:56 - Loading Emmental default config from /dfs/scratch1/senwu/mmtl/emmental/src/emmental/emmental-default-config.yaml.
[2019-05-22 18:34:00,961][INFO] emmental.meta:143 - Updating Emmental config from user provided config.


In [5]:
Meta.config

{'meta_config': {'seed': 111, 'verbose': True, 'log_path': None},
 'model_config': {'model_path': None, 'device': 1, 'dataparallel': False},
 'learner_config': {'fp16': False,
  'n_epochs': 20,
  'train_split': 'train',
  'valid_split': 'val',
  'test_split': 'test',
  'ignore_index': -100,
  'optimizer_config': {'optimizer': 'adam',
   'lr': 1e-05,
   'l2': 0.0,
   'grad_clip': 1.0,
   'sgd_config': {'momentum': 0.9},
   'adam_config': {'betas': (0.9, 0.999)}},
  'lr_scheduler_config': {'lr_scheduler': 'linear',
   'warmup_steps': None,
   'warmup_unit': 'batch',
   'warmup_percentage': 0.1,
   'min_lr': 0.0,
   'linear_config': {'min_lr': 0.0},
   'exponential_config': {'gamma': 0.9},
   'plateau_config': {'factor': 0.5, 'patience': 10, 'threshold': 0.0001}},
  'task_scheduler': 'round_robin',
  'global_evaluation_metric_dict': None,
  'min_lr': 0},
 'logging_config': {'counter_unit': 'epoch',
  'evaluation_freq': 1,
  'writer_config': {'writer': 'tensorboard', 'verbose': True},
  'c

In [6]:
TASK_NAME = "CB"
DATA_DIR = "data"
BERT_MODEL_NAME = "bert-large-cased"
BATCH_SIZE = 4

BERT_OUTPUT_DIM = 768 if "base" in BERT_MODEL_NAME else 1024
TASK_CARDINALITY = (
    len(SuperGLUE_LABEL_MAPPING[TASK_NAME].keys())
    if SuperGLUE_LABEL_MAPPING[TASK_NAME] is not None
    else 1
)

In [7]:
BERT_OUTPUT_DIM, TASK_CARDINALITY

(1024, 3)

# Extract train/dev dataset from file

### Slice function

In [8]:
def slice_base(dataset):
    slice_name = "slice_base"
    return torch.from_numpy(np.array([1] * len(dataset))), dataset.Y_dict["labels"]

In [9]:
def slice_base_1(dataset):
    slice_name = "slice_base_1"
    return torch.from_numpy(np.array([1] * len(dataset))), dataset.Y_dict["labels"]

In [10]:
slice_func_dict = {
    "slice_base": slice_base,
    "slice_base_1": slice_base_1,
}
slice_func_dict.keys()

dict_keys(['slice_base', 'slice_base_1'])

In [11]:
dataloaders = get_CB_dataloaders(
    data_dir=DATA_DIR,
    task_name=TASK_NAME,
    splits=["train", "val", "test"],
    max_sequence_length=256,
    max_data_samples=None,
    tokenizer_name=BERT_MODEL_NAME,
    batch_size=BATCH_SIZE,
    slice_func_dict=slice_func_dict,
)

[2019-05-22 18:34:01,276][INFO] tokenizer:8 - Loading Tokenizer bert-large-cased
[2019-05-22 18:34:01,568][INFO] pytorch_pretrained_bert.tokenization:190 - loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt from cache at /lfs/local/0/senwu/.pytorch_pretrained_bert/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


data/CB/train.jsonl
{'premise': 'It was a complex language. Not written down but handed down. One might say it was peeled down.', 'hypothesis': 'the language was peeled down', 'label': 'entailment', 'idx': 0}


[2019-05-22 18:34:02,125][INFO] parse_CB_slice:133 - Loaded train for CB.
[2019-05-22 18:34:02,256][INFO] parse_CB_slice:133 - Loaded val for CB.


data/CB/val.jsonl
{'premise': "Valence the void-brain, Valence the virtuous valet. Why couldn't the figger choose his own portion of titanic anatomy to shaft? Did he think he was helping?", 'hypothesis': 'Valence was helping', 'label': 'contradiction', 'idx': 0}
data/CB/test.jsonl
{'premise': 'Polly had to think quickly. They were still close enough to shore for him to return her to the police if she admitted she was not an experienced ocean sailor.', 'hypothesis': 'Polly was not an experienced ocean sailor', 'idx': 0}


[2019-05-22 18:34:02,808][INFO] parse_CB_slice:133 - Loaded test for CB.


In [12]:
dataloaders["train"].dataset.Y_dict

{'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
         1, 1, 2, 2, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1,
         1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3,
         3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
         2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
         1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
         2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
         2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1,
         3, 1, 3, 3, 1, 3, 3, 1, 3, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2,
         2, 1, 2, 2, 2, 2, 1, 1, 1, 1]),
 'CB_slice_ind_slice_base': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1

# Build Emmental task

In [13]:
def ce_loss(module_name, immediate_ouput_dict, Y, active):
    return F.cross_entropy(
        immediate_ouput_dict[module_name][0][active], (Y.view(-1) - 1)[active]
    )

In [14]:
def output(module_name, immediate_ouput_dict):
    return F.softmax(immediate_ouput_dict[module_name][0], dim=1)

In [15]:
def macro_f1(golds, probs, preds):
    return {"macro_f1": f1_score(golds, preds, average="macro")}

In [16]:
from emmental.metrics.accuracy import accuracy_scorer

def accuacy_macro_f1(golds, probs, preds):
    metric_dict = macro_f1(golds, probs, preds)
    metric_dict.update(accuracy_scorer(golds, probs, preds))
    metric_dict["accuacy_macro_f1"] = (metric_dict["accuracy"] + metric_dict["macro_f1"]) / 2
    return metric_dict

In [17]:
H = BERT_OUTPUT_DIM
shared_classification_module = nn.Linear(H, TASK_CARDINALITY)
bert_module = BertModule(BERT_MODEL_NAME)

[2019-05-22 18:34:03,360][INFO] pytorch_pretrained_bert.modeling:580 - loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz from cache at ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
[2019-05-22 18:34:03,364][INFO] pytorch_pretrained_bert.modeling:588 - extracting archive file ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233 to temp dir /tmp/tmpmcypep1t
[2019-05-22 18:34:20,819][INFO] pytorch_pretrained_bert.modeling:598 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pooler_fc_size": 768,
  "pooler_num_a

In [18]:
tasks = []

In [19]:
CAT = 1

In [20]:
class FeatureConcateModule(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, feature, idx1=None, idx2=None):
#         import pdb; pdb.set_trace()
        last_layer = feature[-1]
        emb = last_layer[:,0,:]
        
        if idx1 and idx2:
            idx1 = idx1.unsqueeze(-1).unsqueeze(-1).expand([-1, -1, last_layer.size(-1)])
            idx2 = idx2.unsqueeze(-1).unsqueeze(-1).expand([-1, -1, last_layer.size(-1)])
            word1_emb = last_layer.gather(dim=1, index=idx1).squeeze(dim=1)
            word2_emb = last_layer.gather(dim=1, index=idx2).squeeze(dim=1)
            input = torch.cat([emb, word1_emb, word2_emb], dim=-1)
        else:
            input = emb
        return input

In [21]:
# Add ind task

type = "ind"

for slice_name in slice_func_dict.keys():
    task = EmmentalTask(
        name=f"{TASK_NAME}_slice_{type}_{slice_name}",
        module_pool=nn.ModuleDict(
            {
#                 "bert_module": bert_module,
                "feature": FeatureConcateModule(),
                f"{TASK_NAME}_slice_{type}_{slice_name}_head": nn.Linear(
                    CAT * BERT_OUTPUT_DIM, 2
                ),
            }
        ),
        task_flow=[
            {
                "name": "input",
                "module": "bert_module",
                "inputs": [("_input_", "token_ids"), ("_input_", "token_segments")],
            },
            {"name": f"feature", "module": f"feature", "inputs": [("input", 0)]},
            {
                "name": f"{TASK_NAME}_slice_{type}_{slice_name}_head",
                "module": f"{TASK_NAME}_slice_{type}_{slice_name}_head",
                "inputs": [("feature", 0)],
            },
        ],
        loss_func=partial(ce_loss, f"{TASK_NAME}_slice_{type}_{slice_name}_head"),
        output_func=partial(output, f"{TASK_NAME}_slice_{type}_{slice_name}_head"),
        scorer=Scorer(metrics=["accuracy"]),
    )
    tasks.append(task)

[2019-05-22 18:34:52,314][INFO] emmental.task:34 - Created task: CB_slice_ind_slice_base
[2019-05-22 18:34:52,316][INFO] emmental.task:34 - Created task: CB_slice_ind_slice_base_1


In [22]:
# Add ind task

type = "pred"

for slice_name in slice_func_dict.keys():
    task = EmmentalTask(
        name=f"{TASK_NAME}_slice_{type}_{slice_name}",
        module_pool=nn.ModuleDict(
            {
#                 "bert_module": bert_module,
                "feature": FeatureConcateModule(),
                f"{TASK_NAME}_slice_feat_{slice_name}": nn.Linear(
                    CAT * BERT_OUTPUT_DIM, H
                ),
                f"{TASK_NAME}_slice_{type}_{slice_name}_head": shared_classification_module,
            }
        ),
        task_flow=[
            {
                "name": "input",
                "module": "bert_module",
                "inputs": [("_input_", "token_ids"), ("_input_", "token_segments")],
            },
            {
                "name": f"feature",
                "module": f"feature",
                "inputs": [
                    ("input", 0),
                    #                     ("_input_", "sent1_idxs"),
                    #                     ("_input_", "sent2_idxs"),
                ],
            },
            {
                "name": f"{TASK_NAME}_slice_feat_{slice_name}",
                "module": f"{TASK_NAME}_slice_feat_{slice_name}",
                "inputs": [("feature", 0)],
            },
            {
                "name": f"{TASK_NAME}_slice_{type}_{slice_name}_head",
                "module": f"{TASK_NAME}_slice_{type}_{slice_name}_head",
                "inputs": [(f"{TASK_NAME}_slice_feat_{slice_name}", 0)],
            },
        ],
        loss_func=partial(ce_loss, f"{TASK_NAME}_slice_{type}_{slice_name}_head"),
        output_func=partial(output, f"{TASK_NAME}_slice_{type}_{slice_name}_head"),
        scorer=Scorer(customize_metric_funcs={"accuacy_macro_f1": accuacy_macro_f1}),
    )
    tasks.append(task)

[2019-05-22 18:34:52,373][INFO] emmental.task:34 - Created task: CB_slice_pred_slice_base
[2019-05-22 18:34:52,388][INFO] emmental.task:34 - Created task: CB_slice_pred_slice_base_1


In [23]:
class MasterModule(nn.Module):
    def __init__(self, feature_dim, class_cardinality):
        super().__init__()
        self.linear = nn.Linear(feature_dim, class_cardinality)

    def forward(self, immediate_ouput_dict):
        slice_ind_names = sorted(
            [
                flow_name
                for flow_name in immediate_ouput_dict.keys()
                if "_slice_ind_" in flow_name
            ]
        )
        slice_pred_names = sorted(
            [
                flow_name
                for flow_name in immediate_ouput_dict.keys()
                if "_slice_pred_" in flow_name
            ]
        )

        Q = torch.cat(
            [
                F.softmax(immediate_ouput_dict[slice_ind_name][0])[:, 0].unsqueeze(1)
                for slice_ind_name in slice_ind_names
            ],
            dim=-1,
        )
        P = torch.cat(
            [
                F.softmax(immediate_ouput_dict[slice_pred_name][0])[:, 0].unsqueeze(1)
                for slice_pred_name in slice_pred_names
            ],
            dim=-1,
        )

        slice_feat_names = sorted(
            [
                flow_name
                for flow_name in immediate_ouput_dict.keys()
                if "_slice_feat_" in flow_name
            ]
        )

        slice_reps = torch.cat(
            [
                immediate_ouput_dict[slice_feat_name][0].unsqueeze(1)
                for slice_feat_name in slice_feat_names
            ],
            dim=1,
        )

        A = F.softmax(Q * P, dim=1).unsqueeze(-1).expand([-1, -1, slice_reps.size(-1)])

        reweighted_rep = torch.sum(A * slice_reps, 1)

        return self.linear.forward(reweighted_rep)

In [24]:
master_task = EmmentalTask(
    name=f"{TASK_NAME}",
    module_pool=nn.ModuleDict(
        {
            "bert_module": bert_module,
            f"{TASK_NAME}_pred_head": MasterModule(H, TASK_CARDINALITY),
        }
    ),
    task_flow=[
        {
            "name": f"{TASK_NAME}_pred_head",
            "module": f"{TASK_NAME}_pred_head",
            "inputs": [],
        }
    ],
    loss_func=partial(ce_loss, f"{TASK_NAME}_pred_head"),
    output_func=partial(output, f"{TASK_NAME}_pred_head"),
    scorer=Scorer(customize_metric_funcs={"accuacy_macro_f1": accuacy_macro_f1}),
)
tasks.append(master_task)

[2019-05-22 18:34:52,471][INFO] emmental.task:34 - Created task: CB


In [25]:
mtl_model = EmmentalModel(name="SuperGLUE_single_task", tasks=tasks)
emmental_learner = EmmentalLearner()
emmental_learner.learn(mtl_model, dataloaders.values())

[2019-05-22 18:34:52,507][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:57,389][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:57,392][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:57,399][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:57,405][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:58,750][INFO] emmental.model:44 - Created emmental model SuperGLUE_single_task that contains task {'CB_slice_pred_slice_base', 'CB_slice_pred_slice_base_1', 'CB_slice_ind_slice_base', 'CB', 'CB_slice_ind_slice_base_1'}.
[2019-05-22 18:34:58,751][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-22 18:34:58,759][INFO] emmental.logging.logging_manager:33 - Evaluating every 1 epoch.
[2019-05-22 18:34:58,760][INFO] emmental.logging.logging_manager:43 - Checkpointing every 1 epoch.
[2019-05-22 18:34:58,761][INFO] emmental.logging.checkpointer:42 - Save che

/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[2019-05-22 18:35:50,334][INFO] emmental.logging.checkpointer:93 - checkpoint_runway condition has been met. Start checkpoining.
/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type FeatureConcateModule. It won't b

[2019-05-22 18:37:43,207][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 2.0 epoch at logs/2019_05_22/18_34_00/checkpoint_2.0.pth.
[2019-05-22 18:37:55,548][INFO] emmental.logging.checkpointer:119 - Save best model of metric CB/SuperGLUE/val/accuacy_macro_f1 at logs/2019_05_22/18_34_00/best_model_CB_SuperGLUE_val_accuacy_macro_f1.pth


[2019-05-22 18:38:57,869][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 3.0 epoch at logs/2019_05_22/18_34_00/checkpoint_3.0.pth.


[2019-05-22 18:40:00,796][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 4.0 epoch at logs/2019_05_22/18_34_00/checkpoint_4.0.pth.
[2019-05-22 18:40:11,357][INFO] emmental.logging.checkpointer:119 - Save best model of metric CB/SuperGLUE/val/accuacy_macro_f1 at logs/2019_05_22/18_34_00/best_model_CB_SuperGLUE_val_accuacy_macro_f1.pth


...

[2019-05-22 18:54:28,017][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 17.0 epoch at logs/2019_05_22/18_34_00/checkpoint_17.0.pth.


[2019-05-22 18:55:30,226][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 18.0 epoch at logs/2019_05_22/18_34_00/checkpoint_18.0.pth.


[2019-05-22 18:56:33,205][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 19.0 epoch at logs/2019_05_22/18_34_00/checkpoint_19.0.pth.


[2019-05-22 18:57:35,863][INFO] emmental.logging.checkpointer:103 - Save checkpoint of 20.0 epoch at logs/2019_05_22/18_34_00/checkpoint_20.0.pth.
[2019-05-22 18:57:35,917][INFO] emmental.logging.checkpointer:149 - Clear all immediate checkpoints.


[2019-05-22 18:57:43,002][INFO] emmental.logging.checkpointer:189 - Loading the best model from logs/2019_05_22/18_34_00/best_model_CB_SuperGLUE_val_accuacy_macro_f1.pth.
[2019-05-22 18:57:45,314][INFO] emmental.model:58 - Moving model to GPU (cuda:1).


In [26]:
mtl_model.score(dataloaders["val"])

/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'CB_slice_ind_slice_base/SuperGLUE/val/accuracy': 1.0,
 'CB_slice_pred_slice_base/SuperGLUE/val/macro_f1': 0.9237290159275974,
 'CB_slice_pred_slice_base/SuperGLUE/val/accuracy': 0.9285714285714286,
 'CB_slice_pred_slice_base/SuperGLUE/val/accuacy_macro_f1': 0.926150222249513,
 'CB_slice_ind_slice_base_1/SuperGLUE/val/accuracy': 1.0,
 'CB_slice_pred_slice_base_1/SuperGLUE/val/macro_f1': 0.8917213078192603,
 'CB_slice_pred_slice_base_1/SuperGLUE/val/accuracy': 0.9107142857142857,
 'CB_slice_pred_slice_base_1/SuperGLUE/val/accuacy_macro_f1': 0.9012177967667729,
 'CB/SuperGLUE/val/macro_f1': 0.9237290159275974,
 'CB/SuperGLUE/val/accuracy': 0.9285714285714286,
 'CB/SuperGLUE/val/accuacy_macro_f1': 0.926150222249513}